# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298358755087                   -0.85    5.5         
  2   -8.300181819885       -2.74       -1.25    1.0    102ms
  3   -8.300428454089       -3.61       -1.88    2.8    132ms
  4   -8.300461392492       -4.48       -2.75    1.9    114ms
  5   -8.300463826955       -5.61       -3.02    3.0    252ms
  6   -8.300464235403       -6.39       -3.22    5.5    156ms
  7   -8.300464439005       -6.69       -3.37    5.0    174ms
  8   -8.300464551915       -6.95       -3.53    2.0    128ms
  9   -8.300464617040       -7.19       -3.72    1.5    121ms
 10   -8.300464631630       -7.84       -3.86    1.4    120ms
 11   -8.300464642679       -7.96       -4.15    3.0    192ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67386911274                   -0.70    6.4         
  2   -16.67869706351       -2.32       -1.14    1.2    264ms
  3   -16.67920402847       -3.30       -1.87    1.5    270ms
  4   -16.67927957614       -4.12       -2.77    5.0    395ms
  5   -16.67928593309       -5.20       -3.18    3.8    361ms
  6   -16.67928619564       -6.58       -3.48    2.5    314ms
  7   -16.67928621613       -7.69       -3.95    1.9    268ms
  8   -16.67928622146       -8.27       -4.56    2.5    304ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32781589420                   -0.56    6.9         
  2   -33.33458767436       -2.17       -1.00    1.1    991ms
  3   -33.33596664461       -2.86       -1.72    5.1    1.29s
  4   -33.33614054105       -3.76       -2.63    4.4    1.19s
  5   -33.33681582949       -3.17       -2.27    8.0    1.62s
  6   -33.33685052758       -4.46       -2.34    1.2    904ms
  7   -33.33694338622       -4.03       -3.64    1.6    1.02s
  8   -33.33694365386       -6.57       -3.68    5.4    1.60s
  9   -33.33694378035       -6.90       -4.25    1.4    903ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298132698358                   -0.85    5.2         
  2   -8.300253214478       -2.67       -1.59    1.0   82.1ms
  3   -8.300409723682       -3.81       -2.45    3.2    115ms
  4   -8.300348052224   +   -4.21       -2.22    3.2    136ms
  5   -8.300463097170       -3.94       -3.20    1.1   83.9ms
  6   -8.300464479222       -5.86       -3.62    2.6    118ms
  7   -8.300464632305       -6.82       -4.21    1.8   96.4ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.28911271880                   -0.56    6.8         
  2   -33.32991039864       -1.39       -1.26    1.5    811ms
  3   -29.01976558417   +    0.63       -0.75    6.8    1.53s
  4   -31.96609679950        0.47       -0.99    4.6    1.45s
  5   -33.23069096477        0.10       -1.24    3.4    1.20s
  6   -33.23725915560       -2.18       -1.54    2.5    1.07s
  7   -32.61171692003   +   -0.20       -1.13    4.4    1.27s
  8   -33.33102472590       -0.14       -2.09    3.6    1.12s
  9   -33.33426933033       -2.49       -2.24    3.6    1.13s
 10   -33.33510893297       -3.08       -2.30    1.6    889ms
 11   -33.33600337711       -3.05       -2.44    1.8    892ms
 12   -33.33682152550       -3.09      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.